# qa-nlp
Question answering neural model based on the SQuAD dataset.

Authors:
- Lorenzo Mario Amorosa
- Andrea Espis
- Mattia Orlandi
- Giacomo Pinardi

## 0. Environment setup

In [4]:
%load_ext autoreload
%autoreload 2

In [10]:
# Library to read json
import json

# Numeric and data manipulation tools
import pandas as pd
import numpy as np
import random

# Deep learning framework
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Natural language tools
import nltk
from nltk.tokenize import TreebankWordTokenizer
import gensim
import gensim.downloader as gloader

# Other tools
from tqdm.notebook import tqdm
from collections import OrderedDict, Counter
from time import time
from itertools import zip_longest
import collections

# automatic mixed precision training:
from torch.cuda.amp import autocast 
from torch.cuda.amp import GradScaler

# Type hint
from typing import Optional, Callable, Tuple, Dict, List, Union

nltk.download('punkt')
nltk.download('stopwords')

# from sklearn.model_selection import train_test_split

# Use GPU acceleration if possible
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using this device:', DEVICE)

if not(torch.cuda.is_available()):
    raise Exception('Switch to runtime GPU, otherwise the code won\'t work properly')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Using this device: cuda:0


In [2]:
# Set seed for reproducibility
def fix_random(seed: int):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

fix_random(42)

In [3]:
# Use GPU acceleration if possible
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("using this device:", DEVICE)

# Define special tokens
PAD = '<PAD>'
UNK = '<UNK>'

using this device: cuda:0


In [18]:
# Lambda for computing the mean of a list
mean: Callable[[List[float]], float] = lambda l: sum(l) / len(l)

# Lambda for transforming a list of tuples into a tuple of lists
to_tuple_of_lists: Callable[[List[Tuple]], Tuple[List]] = lambda list_of_tuples: tuple(map(list, zip(*list_of_tuples)))

# Lambda for transforming a tuple of lists into a list of tuples
to_list_of_tuples: Callable[[Tuple[List]], List[Tuple]] = lambda tuple_of_lists: list(zip(*tuple_of_lists))

# Lambda for iterating with batches (if the length of the sequences does not match with the batch size, tuples of empty lists are appended)
batch_iteration: Callable[[List[Tuple]], zip] = lambda data, batch_size: zip_longest(*[iter(data)] * batch_size, fillvalue=([], [], []))

## 1. Dataset preparation

In [20]:
"""
json structure:

data []
|---title
|---paragraphs []
|   |---context
|   |---qas []
|   |   |---answers []
|   |   |   |---answer_start
|   |   |   |---text
|   |   |---question
|   |   |---id
version

"""

filename = 'training_set.json'

with open(filename, 'r') as f:
    raw_data = f.readlines()[0]

parsed_data = json.loads(raw_data)['data']

context_list = []
context_index = -1
paragraph_index = -1

dataset = {'paragraph_index': [], 'context_index': [], 'question': [], 'id': [], 'answer_start': [], 'answer_end': [], 'answer_text': []}

for i in range(len(parsed_data)):
    paragraph_index += 1
    for j in range(len(parsed_data[i]['paragraphs'])):
        context_list.append(parsed_data[i]['paragraphs'][j]['context'])
        context_index += 1

        for k in range(len(parsed_data[i]['paragraphs'][j]['qas'])):
            question = parsed_data[i]['paragraphs'][j]['qas'][k]['question']
            id = parsed_data[i]['paragraphs'][j]['qas'][k]['id']

            for l in range(len(parsed_data[i]['paragraphs'][j]['qas'][k]['answers'])): 
                answer_start = parsed_data[i]['paragraphs'][j]['qas'][k]['answers'][l]['answer_start']
                answer_text = parsed_data[i]['paragraphs'][j]['qas'][k]['answers'][l]['text']

                answer_end = answer_start + len(answer_text)

                dataset['paragraph_index'].append(paragraph_index)
                dataset['context_index'].append(context_index)
                dataset['question'].append(question)
                dataset['id'].append(id)
                dataset['answer_start'].append(answer_start)
                dataset['answer_end'].append(answer_end)
                dataset['answer_text'].append(answer_text)

df = pd.DataFrame.from_dict(dataset)

df.head()

,paragraph_index,context_index,question,id,answer_start,answer_end,answer_text
0,0,0,To whom did the Virgin Mary allegedly appear i...,5733be284776f41900661182,515,541,Saint Bernadette Soubirous
1,0,0,What is in front of the Notre Dame Main Building?,5733be284776f4190066117f,188,213,a copper statue of Christ
2,0,0,The Basilica of the Sacred heart at Notre Dame...,5733be284776f41900661180,279,296,the Main Building
3,0,0,What is the Grotto at Notre Dame?,5733be284776f41900661181,381,420,a Marian place of prayer and reflection
4,0,0,What sits on top of the Main Building at Notre...,5733be284776f4190066117e,92,126,a golden statue of the Virgin Mary


In [ ]:
# NOT REALLY NEEDED: evaluate.py accept the full dataset (dataset.json)
# Generating ground truth .json file
# ground_truth = df[['id', 'answer_text']].set_index('id')['answer_text'].to_dict()

# with open("ground_truth.txt", "w") as f:
#    f.write(json.dumps(ground_truth))


In [ ]:
# L'unica cosa che si potrebbe rimuovere è la fonetica, ma è poco presente. Inoltre facendolo bisognerebbe aggiornare gli indici

# Some examples of contexts and questions:
for i in range(0, 4000, 100):
    # print('Title:   ', title_list[df['title_index'][i]])
    print('Context: ', context_list[df['context_index'][i]])
    print('Question:', df['question'][i], "\n")

In [21]:
# Define split ratios
test_ratio = 0.2
val_ratio = 0.2

# Build array of paragraphs indexes and shuffle them
paragraph_indexes = df['paragraph_index'].unique()
np.random.shuffle(paragraph_indexes)
n_samples = len(paragraph_indexes)

# Reserve indexes for test set
test_size = int(test_ratio * n_samples)
train_val_size = n_samples - test_size
test_indexes = paragraph_indexes[-test_size:]
# Reserve indexes for validation set
val_size = int(val_ratio * train_val_size)
train_size = train_val_size - val_size
val_indexes = paragraph_indexes[-(test_size + val_size):-test_size]
# Reserve indexes for training set
train_indexes = paragraph_indexes[:train_size]

assert train_size == len(train_indexes), 'Something went wrong with train set slicing'
assert val_size == len(val_indexes), 'Something went wrong with val set slicing'
assert test_size == len(test_indexes), 'Something went wrong with test set slicing'

print('Number of train paragraphs:', train_size)
print('Number of validation paragraphs:', val_size)
print('Number of test paragraphs:', test_size)

# Split dataframe
df_train = df[np.in1d(df['paragraph_index'], train_indexes)]
df_val = df[np.in1d(df['paragraph_index'], val_indexes)]
df_test = df[np.in1d(df['paragraph_index'], test_indexes)]

print('\nNumber of train samples:', len(df_train))
print('Number of validation samples:', len(df_val))
print('Number of test samples:', len(df_test))

# Obtaining list of ids for accurate performance evaluation
id_train = df_train['id'].to_list()
id_val = df_val['id'].to_list()
id_test = df_test['id'].to_list()

Number of train paragraphs: 284
Number of validation paragraphs: 70
Number of test paragraphs: 88

Number of train samples: 57046
Number of validation samples: 13113
Number of test samples: 17440


## 2. Embeddings

In [8]:
print('Downloading GloVe model...')
emb_dim = 50
glove_model = gloader.load('glove-wiki-gigaword-' + str(emb_dim))
print('\nDownload completed.')

[==================================================] 100.0% 66.0/66.0MB downloaded

Download completed.


In [23]:
if True:
    df_train = df_train[:10000]
    df_val = df_val[:3000]
    df_test = df_test[:100]

In [24]:
def tokenize_corpus(df: pd.DataFrame, context_list: List[str]):
    twt = TreebankWordTokenizer()
    
    t_start = time()
    # Retrieve contexts
    contexts = df['context_index'].apply(lambda x: context_list[x])
    # Tokenize both contexts and queries
    x_ctx = contexts.apply(lambda x: twt.tokenize(x)).tolist()
    x_qry = df['question'].apply(lambda x: twt.tokenize(x)).tolist()
    # Get indexes to start_end characters
    y_char = [(start, end) for start, end in zip(df['answer_start'].tolist(), df['answer_end'].tolist())]
    # Get spans of tokens
    spans_list = contexts.apply(lambda x: twt.span_tokenize(x)).tolist()
    # Convert indexes s.t. the point to start/end tokens
    y = []
    for spans, (char_start, char_end) in zip(spans_list, y_char):
        token_start, token_end = None, None
        for i, span in enumerate(spans):
            if span[0] <= char_start <= span[1]:
                token_start = i
            if span[0] <= char_end <= span[1]:
                token_end = i
        y.append((token_start, token_end))
    print(f'[{time() - t_start:.3f} s]')
    
    return x_ctx, x_qry, y

def build_vocabulary(corpus: List[List[str]],
                     old_word_listing: Optional[List[str]] = None) -> (Dict[int, str], Dict[int, str], List[str]):
    flat_tokens = [x for sub in corpus for x in sub]
    
    if old_word_listing is None:  # standard case
        word_listing = [PAD] + list(OrderedDict.fromkeys(flat_tokens))
    else:  # case in which we extend an already existing vocabulary
        word_listing = list(OrderedDict.fromkeys(old_word_listing + flat_tokens))
        
    idx_to_word = {i: w for i, w in enumerate(word_listing)}
    word_to_idx = {w: i for i, w in enumerate(word_listing)}

    return idx_to_word, word_to_idx, word_listing

# Tokenize corpus
print('Tokenizing training corpus...', end=' ')
X_trainC, X_trainQ, Y_train = tokenize_corpus(df_train, context_list)
train_corpus = X_trainC + X_trainQ

print('Tokenizing validation corpus...', end=' ')
X_valC, X_valQ, Y_val = tokenize_corpus(df_val, context_list)
val_corpus = X_valC + X_valQ

print('Tokenizing test corpus...', end=' ')
X_testC, X_testQ, Y_test = tokenize_corpus(df_test, context_list)
test_corpus = X_testC + X_testQ

# Get word and char mappings for each set
train_i2w, train_w2i, train_wl = build_vocabulary(train_corpus)
val_i2w, val_w2i, val_wl = build_vocabulary(val_corpus, train_wl)
test_i2w, test_w2i, test_wl = build_vocabulary(test_corpus, val_wl)

print('-' * 50)
print('Words in training set:', len(train_wl))
print('Words in validation set:', len(val_wl))
print('Words in test set:', len(test_wl))

Tokenizing training corpus... 

ValueError: ignored

In [ ]:
train_oov_words = [word for word in train_wl if word not in glove_model.vocab and word != PAD]
val_oov_words = [word for word in val_wl if word not in glove_model.vocab and word != PAD]
test_oov_words = [word for word in test_wl if word not in glove_model.vocab and word != PAD]

print(f'Total OOV terms in training set: {len(train_oov_words)} ({float(len(train_oov_words)) / len(train_wl) * 100:.2f}%)')
print(f'Total OOV terms in validation set: {len(val_oov_words)} ({float(len(val_oov_words)) / len(val_wl) * 100:.2f}%)')
print(f'Total OOV terms in test set: {len(test_oov_words)} ({float(len(test_oov_words)) / len(test_wl) * 100:.2f}%)')

Total OOV terms in training set: 14167 (48.23%)
Total OOV terms in validation set: 17909 (49.73%)
Total OOV terms in test set: 17980 (49.80%)


In [ ]:
def build_word_embedding_matrix(embedding_model: gensim.models.keyedvectors.Word2VecKeyedVectors,
                                word_to_idx: Dict[str, int],
                                oov_words: List[str],
                                old_word_embedding_matrix: Optional[np.ndarray] = None):
    # Initialize embedding matrix with all zeros
    embedding_matrix = np.zeros((len(word_to_idx), embedding_model.vector_size))
    
    # Analyze embeddings to get mean and standard deviation
    mean_list, std_list = [], []
    for word in tqdm(word_to_idx.keys(), leave=False):
        if word not in oov_words and word != PAD:
            embed = embedding_model[word]
            # Compute mean and std
            mean_list.append(np.mean(embed))
            std_list.append(np.std(embed))

    embedding_mean = mean(mean_list)
    embedding_std = mean(std_list)

    for word, idx in tqdm(word_to_idx.items(), leave=False):
        # If word is PAD no action is performed (it will be assigned the zero vector)
        if word not in oov_words and word != PAD:
            embedding_matrix[idx] = embedding_model[word]
        elif word in oov_words:
            oov_idx = word_to_idx[word]
            if old_word_embedding_matrix is None or oov_idx >= len(old_word_embedding_matrix):
                embedding_matrix[idx] = np.random.normal(loc=embedding_mean, scale=embedding_std, size=embedding_model.vector_size)
            else:
                embedding_matrix[idx] = old_word_embedding_matrix[oov_idx]
            
    return embedding_matrix

# Build word embedding matrix based only on the training set (for training)
train_emb_mtx = build_word_embedding_matrix(glove_model, train_w2i, train_oov_words)
print('Shape of word embedding matrix (training set):', train_emb_mtx.shape)

# Build word embedding matrix based on training + validation set (for validation)
val_emb_mtx = build_word_embedding_matrix(glove_model, val_w2i, val_oov_words, train_emb_mtx)
print('Shape of word embedding matrix (validation set):', val_emb_mtx.shape)

# Build word embedding matrix based on training + validation + test set (for test)
test_emb_mtx = build_word_embedding_matrix(glove_model, test_w2i, test_oov_words, val_emb_mtx)
print('Shape of word embedding matrix (test set):', test_emb_mtx.shape)

Shape of word embedding matrix (training set): (29372, 50)


Shape of word embedding matrix (validation set): (36014, 50)


Shape of word embedding matrix (test set): (36107, 50)


In [ ]:
def build_char_embedding_matrix(corpus: List[str],
                                enc_dim: Optional[int] = 100):
    # Flatten to obtain single characters
    flat_chars = [c for sent in corpus for word in sent for c in word]
    
    # Sort characters by occurrences
    unique_chars = Counter(flat_chars)
    char_listing = sorted(unique_chars, key=unique_chars.get, reverse=True)
    # Select only the enc_dim most frequent ones
    if len(char_listing) > enc_dim - 1:
        char_listing = char_listing[:enc_dim - 1]
    char_listing = [PAD] + char_listing + [UNK]  # add PAD and UNK tokens
    
    idx_to_char = {i: c for i, c in enumerate(char_listing)}
    char_to_idx = {c: i for i, c in enumerate(char_listing)}
    
    # Create one-hot vectors, reserving the last one for UNK (0...0, 1)
    one_hot_chars = np.zeros((len(char_listing) - 1, enc_dim))
    np.fill_diagonal(one_hot_chars, 1)
    one_hot_chars = np.vstack([np.zeros((1, enc_dim)), one_hot_chars])  # stack zero vector on top for PAD
    
    return idx_to_char, char_to_idx, char_listing, one_hot_chars

# Build char embedding matrix based only on the training set, and use it for validation and test too:
# in fact, we can assume that characters appear uniformly in the three splits;
# for those rare case in which this does not happen, we assign the UNK vector
i2c, c2i, cl, char_emb_mtx = build_char_embedding_matrix(train_corpus)
print('Shape of char embedding matrix (training set):', char_emb_mtx.shape)

Shape of char embedding matrix (training set): (101, 100)


## 3. Training


In [8]:
def compute_f1(true_answer, predicted_answer):
    common = collections.Counter(true_answer) & collections.Counter(predicted_answer)
    
    num_same = sum(common.values())

    if num_same == 0:
        return 0
    
    precision = 1.0 * num_same / len(predicted_answer)
    recall = 1.0 * num_same / len(true_answer)
    f1 = (2 * precision * recall) / (precision + recall)

    return f1

In [12]:
def get_raw_scores (context: Tuple[List[str]],
                    label_start: List[int],
                    label_end: List[int],
                    p_start: List[int],
                    p_end: List[int]):
    
    exact_scores = []
    f1_scores = []

    for i, c in enumerate(context):
        true_answer = c[label_start[i]:label_end[i]]
        predicted_answer = c[p_start[i]:p_end[i]]

        exact_scores.append(int(true_answer == predicted_answer))
        f1_scores.append(compute_f1(true_answer, predicted_answer))

    return exact_scores, f1_scores

In [ ]:
def train(model: BiDAF,
          data: List[Tuple[List[str], List[str], str]],
          batch_size: int,
          criterion: Callable[[torch.Tensor, torch.Tensor], float],
          optimizer: torch.optim,
          tensor_maker: TensorMaker,
          max_len: Optional[int] = None,
          verbose: Optional[bool] = False,
          scaler: Optional[torch.cuda.amp.grad_scaler.GradScaler] = False) -> (float, float, float):
    loss_data = []
    distance_start = 0
    distance_end = 0
    total = 0
    flag = 0
    
    # Create batch iterator
    batch_iter = batch_iteration(data, batch_size)
    steps = len(data) // batch_size if len(data) % batch_size == 0 else len(data) // batch_size + 1
    if verbose:
        batch_iter = tqdm(batch_iter, total=steps, leave=False)
    for batch in batch_iter:
        batch_context, batch_query, batch_label = to_tuple_of_lists(batch)
    
        # Filter valid samples in batches (in case of incomplete ones)
        batch_context = [c for c in batch_context if len(c) > 0]
        batch_query = [q for q in batch_query if len(q) > 0]
        batch_label = [l for l in batch_label if len(l) > 0]

        # Extract start and end indexes
        labels_start, labels_end = to_tuple_of_lists(batch_label)

        total_batch = len(batch_context)
        
        with torch.autograd.set_detect_anomaly(True):

            context_word_tensor, context_char_tensor, context_lengths = train_tensor_maker.get_tensor(batch_context)
            query_word_tensor, query_char_tensor, query_lengths = train_tensor_maker.get_tensor(batch_query)
            labels_start = torch.cuda.LongTensor(labels_start)
            labels_end = torch.cuda.LongTensor(labels_end)

            if scaler:

                # Make prediction
                optimizer.zero_grad()
                with autocast():   #https://pytorch.org/docs/stable/notes/amp_examples.html
                    p_soft_start, p_soft_end = model(context_word_tensor, context_char_tensor, query_word_tensor, query_char_tensor)
                    loss = criterion(p_soft_start, p_soft_end, labels_start, labels_end)
                # Backpropagation

                scaler.scale(loss).backward()  #https://pytorch.org/docs/stable/notes/amp_examples.html
                scaler.step(optimizer) #https://pytorch.org/docs/stable/notes/amp_examples.html
                scaler.update() #https://pytorch.org/docs/stable/notes/amp_examples.html

            else:
                # Make prediction
                optimizer.zero_grad()
                p_soft_start, p_soft_end = model(context_word_tensor, context_char_tensor, query_word_tensor, query_char_tensor)
                loss = criterion(p_soft_start, p_soft_end, labels_start, labels_end)
                
                p_start = torch.argmax(p_soft_start, dim=1)
                p_end = torch.argmax(p_soft_end, dim=1)

                # Compute distance metric
                start_dist = torch.abs(p_start - labels_start).sum()
                end_dist = torch.abs(p_end - labels_end).sum()

                # Backpropagation
                loss.backward()
                optimizer.step()

            # Obtaining raw scores from from the prediction
            exact_scores, f1_scores = get_raw_scores(batch_context, labels_start, labels_end, p_start, p_end)

            # Update history
            loss_data.append(loss.item())
            distance_start += start_dist.item()
            distance_end += end_dist.item()
            total += total_batch
        
        if flag == 0:
            print(f'Start_pred: {p_start[0].item()}, End_pred: {p_end[0].item()}, Start_true: {labels_start[0].item()}, End_true: {labels_end[0].item()}')
            flag += 1
        
    return mean(loss_data), distance_start / total, distance_end / total

def evaluate(model: nn.Module,
             data: List[Tuple[List[str], List[str], str]],
             batch_size: int,
             criterion: Callable[[torch.Tensor, torch.Tensor], float],
             tensor_maker: TensorMaker,
             label_to_idx: Dict[str, int],
             max_len: Optional[int] = None,
             verbose: Optional[bool] = False) -> (float, float, float):
    loss_data = []
    correct = 0
    total = 0
    
    with torch.no_grad():
        # Create batch iterator
        batch_iter = batch_iteration(data, batch_size)
        steps = len(data) // batch_size if len(data) % batch_size == 0 else len(data) // batch_size + 1
        if verbose:
            batch_iter = tqdm(batch_iter, total=steps, leave=False)
        for batch in batch_iter:
            # Extract samples
            (claims, evidences, labels, ids) = to_tuple_of_lists(batch)
            
            # Get encoding of claims, evidences and labels, and move them to GPU if available
            claims_tensor, claims_lengths = embedder.get_word_embedding(claims, max_len)
            evidences_tensor, evidences_lengths = embedder.get_word_embedding(evidences, max_len)
            labels_tensor = torch.cuda.FloatTensor([label_to_idx[l] for l in labels if len(l) > 0], device = DEVICE)
            
            # Make prediction
            scores = model(claims_tensor, evidences_tensor, claims_lengths, evidences_lengths)

            # Compute loss
            loss = criterion(scores, labels_tensor.unsqueeze(dim=1))
            
            # Compute accuracy
            total_batch = scores.shape[0]
            scores = torch.sigmoid(scores)
            predictions = scores.squeeze() > 0.5
            correct_batch = (predictions == labels_tensor).sum().item()
        
            # Update history
            loss_data.append(loss.item())
            correct += correct_batch
            total += total_batch
              
    return mean(loss_data), correct / total * 100
    

def training_loop(model: nn.Module,
                  train_data: List[Tuple[List[str], List[str], str]],
                  optimizer: torch.optim,
                  epochs: int,
                  batch_size: int,
                  criterion: Callable[[torch.Tensor, torch.Tensor], float],
                  tensor_maker: TensorMaker,
                  val_embedder: Optional[TensorMaker] = None,
                  max_len: Optional[int] = None,
                  lr_scheduler: torch.optim.lr_scheduler = None,
                  val_data: Optional[List[Tuple[List[str], List[str], str, str]]] = None,
                  early_stopping: Optional[bool] = False,
                  patience: Optional[int] = 5,
                  tolerance: Optional[float] = 1e-4,
                  checkpoint_path: Optional[str] = None,
                  verbose: Optional[bool] = True,
                  seed: Optional[int] = 42,
                  mix_scale: Optional[bool] = False) -> (Dict[str, List[float]]):
    # Set seed for reproducibility
    if seed:
        random.seed(seed)

    history = {'loss': [],
               'distance_start': [],
               'distance_end': [],
               'val_loss': [],
               'val_accuracy': []}
    
    # Initialize variables for early stopping (if required)
    if val_data and val_embedder and early_stopping:
        min_val_loss = np.inf
        no_improve_counter = 0
    
    scaler = GradScaler() if mix_scale else False   # https://pytorch.org/docs/stable/notes/amp_examples.html
    
    for ep in range(epochs):
        if verbose:
            print('-' * 100)
            print(f'Epoch {ep + 1}/{epochs}')
        
        # Shuffle training set at each epoch
        random.shuffle(train_data)
        
        start = time()
        train_loss, train_distance_start, train_distance_end = train(model, train_data, batch_size, criterion, optimizer, tensor_maker, max_len, verbose, scaler)
        end = time()
        
        history['loss'].append(train_loss)
        history['distance_start'].append(train_distance_start)
        history['distance_end'].append(train_distance_end)
        if verbose:
            print(f'\tLoss: {train_loss:.5f} - Distance Start: {train_distance_start:.2f} - Distance End: {train_distance_end:.2f} [Time elapsed: {end - start:.2f} s]')
        
        # Do validation if required
        if val_data and val_embedder:
            
            # Shuffle validation set at each epoch
            random.shuffle(val_data)
            
            start = time()
            val_loss, val_accuracy = evaluate(model, val_data, batch_size, criterion, val_embedder, label_to_idx, max_len, verbose)
            end = time()
            
            history['val_loss'].append(val_loss)
            history['val_accuracy'].append(val_accuracy)
            if verbose:
                print(f'\tValidation loss: {val_loss:.5f} - Validation accuracy: {val_accuracy:.2f}% [Time elapsed: {end - start:.2f} s]')
            
            if early_stopping and checkpoint_path:
                # If validation loss is lower than minimum, update minimum
                if val_loss < min_val_loss - tolerance:
                    min_val_loss = val_loss
                    no_improve_counter = 0
                    
                    # Save model
                    torch.save(model.state_dict(), checkpoint_path)
                # otherwise increment counter
                else:
                    no_improve_counter += 1
                # If loss did not improve for 'patience' epochs, break
                if no_improve_counter == patience:
                    if verbose:
                        print(f'Early stopping: no improvement in validation loss for {patience} epochs from {min_val_loss:.5f}')
                    # Restore model to best
                    model.load_state_dict(torch.load(checkpoint_path))
                    model.eval()
                    break
        
        # If lr scheduling is used, invoke next step
        if lr_scheduler:
            lr_scheduler.step()
    
    return history

from utils.train_utils import training_loop

In [ ]:
from model.bidaf import BiDAF
from model.char_embedder import CharEmbedder
from model.word_embedder import WordEmbedder
from model.tensor_maker import TensorMaker

char_embedder = CharEmbedder(init_emb = torch.FloatTensor(char_emb_mtx),
                             out_char_emb_dim = emb_dim,
                             hidden_dim = 64,
                             input_channels = 1,
                             output_channels = 100,
                             kernel_height = 5,
                             trainable = False)

train_word_embedder = WordEmbedder(init_emb = torch.FloatTensor(train_emb_mtx))
val_word_embedder = WordEmbedder(init_emb = torch.FloatTensor(val_emb_mtx))

model_bidaf = BiDAF(char_embedder, train_word_embedder, val_word_embedder, use_constraint = False).to(DEVICE)

train_tensor_maker = TensorMaker(train_w2i, c2i, device=DEVICE)
val_tensor_maker = TensorMaker(val_w2i, c2i, device=DEVICE)

In [ ]:
from utils.bidaf_loss import bidaf_loss

train_data = to_list_of_tuples((X_trainC, X_trainQ, Y_train))
val_data = to_list_of_tuples((X_valC, X_valQ, Y_val))

EP = 100
BS = 32

optimizer = torch.optim.Adam(model_bidaf.parameters(), lr=5e-3)
#optimizer = torch.optim.Adadelta(model_bidaf.parameters(), lr=0.5, rho=0.999, eps=1e-06, weight_decay=0) # slower in time and in loss
criterion = bidaf_loss

history = training_loop(model=model_bidaf,
                        train_data=train_data,
                        optimizer=optimizer,
                        epochs=EP,
                        batch_size=BS,
                        criterion=criterion,
                        train_tensor_maker=train_tensor_maker,
                        val_tensor_maker=val_tensor_maker,
                        val_data=val_data,
                        early_stopping=False,
                        patience = 15,
                        checkpoint_path='bidaf_baseline.pt')

# trying the constraint:

In [ ]:
from model.bidaf import BiDAF
from model.char_embedder import CharEmbedder
from model.word_embedder import WordEmbedder
from model.tensor_maker import TensorMaker

char_embedder = CharEmbedder(init_emb = torch.FloatTensor(char_emb_mtx),
                             out_char_emb_dim = emb_dim,
                             hidden_dim = 64,
                             input_channels = 1,
                             output_channels = 100,
                             kernel_height = 5,
                             trainable = False)

train_word_embedder = WordEmbedder(init_emb = torch.FloatTensor(train_emb_mtx))
val_word_embedder = WordEmbedder(init_emb = torch.FloatTensor(val_emb_mtx))

model_bidaf = BiDAF(char_embedder, train_word_embedder, val_word_embedder, use_constraint = True).to(DEVICE)

train_tensor_maker = TensorMaker(train_w2i, c2i, device=DEVICE)
val_tensor_maker = TensorMaker(val_w2i, c2i, device=DEVICE)

In [ ]:
from utils.bidaf_loss import bidaf_loss

train_data = to_list_of_tuples((X_trainC, X_trainQ, Y_train))
val_data = to_list_of_tuples((X_valC, X_valQ, Y_val))

EP = 100
BS = 32

optimizer = torch.optim.Adam(model_bidaf.parameters(), lr=5e-3)
#optimizer = torch.optim.Adadelta(model_bidaf.parameters(), lr=0.5, rho=0.999, eps=1e-06, weight_decay=0) # slower in time and in loss
criterion = bidaf_loss

history = training_loop(model=model_bidaf,
                        train_data=train_data,
                        optimizer=optimizer,
                        epochs=EP,
                        batch_size=BS,
                        criterion=criterion,
                        train_tensor_maker=train_tensor_maker,
                        val_tensor_maker=val_tensor_maker,
                        val_data=val_data,
                        early_stopping=False,
                        patience = 15,
                        checkpoint_path='bidaf_baseline.pt')

----------------------------------------------------------------------------------------------------
Epoch 1/100


Start (p): 63, End (p): 12, Start (T): 44, End (T): 45
	Loss: 7.42227 - Distance start: 37 - Distance end: 37 [Time elapsed: 38.46 s]


Start (p): 24, End (p): 7, Start (T): 6, End (T): 7
	Validation loss: 6.91218 - Distance start: 35.00 - Distance end: 34.00 [Time elapsed: 4.29 s]
----------------------------------------------------------------------------------------------------
Epoch 2/100


Start (p): 69, End (p): 48, Start (T): 78, End (T): 81
	Loss: 6.51461 - Distance start: 35 - Distance end: 35 [Time elapsed: 37.54 s]


Start (p): 8, End (p): 8, Start (T): 164, End (T): 166
	Validation loss: 6.37157 - Distance start: 33.00 - Distance end: 34.00 [Time elapsed: 4.31 s]
----------------------------------------------------------------------------------------------------
Epoch 3/100


...

Start (p): 26, End (p): 28, Start (T): 0, End (T): 0
	Validation loss: 7.27565 - Distance start: 22.00 - Distance end: 23.00 [Time elapsed: 4.38 s]
----------------------------------------------------------------------------------------------------
Epoch 99/100


Start (p): 21, End (p): 22, Start (T): 21, End (T): 22
	Loss: 2.86678 - Distance start: 9 - Distance end: 10 [Time elapsed: 37.12 s]


Start (p): 74, End (p): 38, Start (T): 57, End (T): 61
	Validation loss: 7.08554 - Distance start: 22.00 - Distance end: 23.00 [Time elapsed: 4.34 s]
----------------------------------------------------------------------------------------------------
Epoch 100/100


Start (p): 76, End (p): 77, Start (T): 72, End (T): 77
	Loss: 2.70011 - Distance start: 9 - Distance end: 9 [Time elapsed: 37.19 s]


Start (p): 52, End (p): 32, Start (T): 60, End (T): 63
	Validation loss: 7.16599 - Distance start: 22.00 - Distance end: 22.00 [Time elapsed: 4.31 s]
